In [ ]:
import openai, time, logging, os
from dotenv import load_dotenv

# Load environment variables from .env file
load_dotenv()

# Retrieve API key from environment variable
api_key = os.getenv("OPENAI_API_KEY")

# Set the API key for the openai module
openai.api_key = api_key

In [ ]:
model='gpt-4-turbo'

# Hardcoded assistant and thread IDs
assistant_id = 'asst_qVmBNF4oJSHXt2PIwmcQGKus'
thread_id = 'thread_ALbVkq1aTsobawtiG3NC1SvA'

In [ ]:
'''
# Create an assistant
assistant = openai.beta.assistants.create(
  name="Solc-verify specifications generator specialist",
  instructions="You help generate formal specifications for smart contracts on the Ethereum network, written in Solidity and which will be verified in the solc-verify verifier.",
  tools=[{"type": "file_search"}],
  model=model,
)

print(assistant.id)
'''

In [ ]:
'''
# Create a thread
thread = openai.beta.threads.create(
    messages=[
        {
            "role": "user",
            "content": "You will help to generate formal specifications for smart contracts on the Ethereum network, written in Solidity and which will be verified in the solc-verify verifier, using exactly the solc-verify annotatation language syntax."
        }
    ]
)

print(thread.id)
'''

In [ ]:
# Create a Message
message = ''' 
Return just the Solidity code for the following ERC20 contract with the following specifications, becareful with the syntax of the solc-verify annotation language and the variables names.

// SPDX-License-Identifier: MIT
pragma solidity >=0.5.7;

/// @notice  invariant  _totalSupply  ==  __verifier_sum_uint(_balances)
contract ERC20 {

    mapping (address => uint) _balances;
    mapping (address => mapping (address => uint)) _allowed;
    uint public _totalSupply;

    event Transfer(address indexed from, address indexed to, uint value);
    event Approval(address indexed owner, address indexed spender, uint value);


    /// @notice  postcondition ( ( _balances[msg.sender] ==  __verifier_old_uint (_balances[msg.sender] ) - value  && msg.sender  != to ) ||   ( _balances[msg.sender] ==  __verifier_old_uint ( _balances[msg.sender]) && msg.sender  == to ) &&  success )   || !success
    /// @notice  postcondition ( ( _balances[to] ==  __verifier_old_uint ( _balances[to] ) + value  && msg.sender  != to ) ||   ( _balances[to] ==  __verifier_old_uint ( _balances[to] ) && msg.sender  == to )  )   || !success
    function transfer(address to, uint value) public returns (bool success);

    /// @notice  postcondition ( ( _balances[from] ==  __verifier_old_uint (_balances[from] ) - value  &&  from  != to ) || ( _balances[from] ==  __verifier_old_uint ( _balances[from] ) &&  from == to ) && success ) || !success 
    /// @notice  postcondition ( ( _balances[to] ==  __verifier_old_uint ( _balances[to] ) + value  &&  from  != to ) || ( _balances[to] ==  __verifier_old_uint ( _balances[to] ) &&  from  == to ) && success ) || !success 
    /// @notice  postcondition ( _allowed[from ][msg.sender] ==  __verifier_old_uint (_allowed[from ][msg.sender] ) - value && success) || ( _allowed[from ][msg.sender] ==  __verifier_old_uint (_allowed[from ][msg.sender]) && !success) ||  from  == msg.sender
    /// @notice  postcondition  _allowed[from ][msg.sender]  <= __verifier_old_uint (_allowed[from ][msg.sender] ) ||  from  == msg.sender
    function transferFrom(address from, address to, uint value) public returns (bool success);

    /// @notice  postcondition (_allowed[msg.sender ][ spender] ==  value  &&  success) || ( _allowed[msg.sender ][ spender] ==  __verifier_old_uint ( _allowed[msg.sender ][ spender] ) && !success )    
    function approve(address spender, uint value) public returns (bool success);

    /// @notice postcondition _balances[owner] == balance
    function balanceOf(address owner) public view returns (uint balance);

      /// @notice postcondition _allowed[owner][spender] == remaining
    function allowance(address owner, address spender) public view returns (uint remaining);
}
'''
message = openai.beta.threads.messages.create(
    thread_id=thread_id,
    role="user",
    content=message
)

In [ ]:
# Run the assistant
run = openai.beta.threads.runs.create(
    thread_id=thread_id,
    assistant_id=assistant_id,
    instructions="Given a ERC20 token solidity code as input, return the exact same contract with the functions annotations in the format accepted by the solc-verify smart contract verifier, just this, nothing more.",
)

def wait_for_run_completion(client, thread_id, run_id, sleep_interval=5):
    """

    Wait for the run to complete, print the elapsed time, and save the response to a file.
    :param client: The OpenAI client.
    :param thread_id: The ID of the thread.
    :param run_id: The ID of the run.
    :param sleep_interval: Time in seconds to wait between checks.
    """
    while True:
        try:
            run = client.beta.threads.runs.retrieve(thread_id=thread_id, run_id=run_id)
            if run.completed_at:
                elapsed_time = run.completed_at - run.created_at
                formatted_elapsed_time = time.strftime(
                    "%H:%M:%S", time.gmtime(elapsed_time)
                )
                print(f"Run completed in {formatted_elapsed_time}")
                logging.info(f"Run completed in {formatted_elapsed_time}")
                # Get messages here once Run is completed!
                messages = client.beta.threads.messages.list(thread_id=thread_id)
                last_message = messages.data[0]
                response = last_message.content[0].text.value
                # print(f"Assistant Response: {response}")
                # save the response to a incrementing file with the user message at the top
                with open(f"response_{run_id}.txt", "w") as f:
                    f.write(f"User Message: {last_message.content[0].text.value}\n\n")
                    f.write(f"Assistant Response: {response}")
                break
        except Exception as e:
            logging.error(f"An error occurred while retrieving the run: {e}")
            break
        logging.info("Waiting for run to complete...")
        time.sleep(sleep_interval)

In [ ]:
wait_for_run_completion(client=openai, thread_id=thread_id, run_id=run.id)

#run_steps = client.beta.threads.runs.steps.list(thread_id=thread_id, run_id=run.id)
#print(f"Steps---> {run_steps.data[0]}")